Following code is used to generate and process the database 
I have provided 2 processed data sets so donot run the following codes unless you want to create a new one

In [1]:
import os
import numpy as np
import cv2
import mediapipe as mp
import pandas as pd 
import time

In [27]:
# This script captures images of the user's hand in different gestures and saves them in the DATA folder that is later used to train the model.

data_folder_path = "./DATA"
if not os.path.exists(data_folder_path):
    os.makedirs(data_folder_path)

no_of_folders = 26
no_of_images = 50

new_width = 320
new_height = 240

cap = cv2.VideoCapture(0)

for folder in range(no_of_folders):
    folder_path = os.path.join(data_folder_path, chr(65 + folder))
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    image_count = 0
    while image_count < no_of_images:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (new_width, new_height), interpolation=cv2.INTER_AREA)
        cv2.putText(frame, "Press 'q' to capture image", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow('Captured_Frame', frame)

        if cv2.waitKey(5) == ord('q'):
            cv2.imwrite(os.path.join(folder_path, str(image_count) + '.jpg'), frame)
            image_count += 1
            print("Image", image_count, "captured")

    print("Folder", chr(65 + folder), "completed")
    time.sleep(2)  

cap.release()
cv2.destroyAllWindows()

Image 1 captured
Image 2 captured
Image 3 captured
Image 4 captured
Image 5 captured
Image 6 captured
Image 7 captured
Image 8 captured
Image 9 captured
Image 10 captured
Image 11 captured
Image 12 captured
Image 13 captured
Image 14 captured
Image 15 captured
Image 16 captured
Image 17 captured
Image 18 captured
Image 19 captured
Image 20 captured
Image 21 captured
Image 22 captured
Image 23 captured
Image 24 captured
Image 25 captured
Image 26 captured
Image 27 captured
Image 28 captured
Image 29 captured
Image 30 captured
Image 31 captured
Image 32 captured
Image 33 captured
Image 34 captured
Image 35 captured
Image 36 captured
Image 37 captured
Image 38 captured
Image 39 captured
Image 40 captured
Image 41 captured
Image 42 captured
Image 43 captured
Image 44 captured
Image 45 captured
Image 46 captured
Image 47 captured
Image 48 captured
Image 49 captured
Image 50 captured
Folder A completed
Image 1 captured
Image 2 captured
Image 3 captured
Image 4 captured
Image 5 captured
Image

In [ ]:
# This script processes the images captured in the previous script and extracts the hand landmarks using the MediaPipe library. The landmarks are then saved in a CSV file.


mp_hands=mp.solutions.hands
mp_drawing=mp.solutions.drawing_utils

data_folder_path = "./DATA"
processed_data_folder_path = "./PROCESSED_DATA"

if not os.path.exists(processed_data_folder_path):
    os.makedirs(processed_data_folder_path)

no_of_folders = 26
no_of_images = 50

for folder in range(no_of_folders):
    if not os.path.exists(os.path.join(processed_data_folder_path,chr(65+folder))):
        os.makedirs(os.path.join(processed_data_folder_path,chr(65+folder)))

hand_landmarks_array = np.array([[]])

for folder in range(no_of_folders):
    for image_count in range(no_of_images):
        picture=cv2.imread(os.path.join(data_folder_path,chr(65+folder),str(image_count)+'.jpg'))
        if picture is not None:
            picture_rgb=cv2.cvtColor(picture,cv2.COLOR_BGR2RGB)

            results=mp_hands.Hands(static_image_mode=True,min_detection_confidence=0.5,min_tracking_confidence=0.5).process(picture_rgb)
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(picture, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    hand_landmarks_array_row = np.array([])

                    for lm in hand_landmarks.landmark:
                        hand_landmarks_array_row = np.append(hand_landmarks_array_row, [lm.x, lm.y, lm.z])
                    hand_landmarks_array_row = np.append(hand_landmarks_array_row, [chr(65+folder)])
                    #print(hand_landmarks_array_row)
                if hand_landmarks_array.size == 0:
                    hand_landmarks_array = np.array([hand_landmarks_array_row])
                else:
                    hand_landmarks_array = np.vstack([hand_landmarks_array, hand_landmarks_array_row])

                cv2.imwrite(os.path.join(processed_data_folder_path,chr(65+folder),str(image_count)+'.jpg'),picture)
            #cv2.imshow('Hands',picture)
            #cv2.waitKey(5)
        else:
            print(f"Image {os.path.join(data_folder_path,chr(65+folder),str(image_count)+'.jpg')} not found or is empty.")
    
df = pd.DataFrame(hand_landmarks_array)
df.to_csv('hand_landmarks.csv', index=False)
cv2.destroyAllWindows()